In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
! pip install numpy
! pip install pymorphy2[fast]
! pip install nltk


     |████████████████████████████████| 55 kB 2.1 MB/s 
     |████████████████████████████████| 8.2 MB 10.5 MB/s 
     |████████████████████████████████| 371 kB 45.3 MB/s 
  Created wheel for DAWG: filename=DAWG-0.8.0-cp37-cp37m-linux_x86_64.whl size=871450 sha256=575517fcae0bdc057837b487886a96229f756a12a4604b11e3de72485083dfaa
  Stored in directory: /root/.cache/pip/wheels/85/51/a4/2de41ff197786537075027c27b479a38da92f50abc86634445
Successfully built DAWG


In [3]:
import random
import nltk
import re
import string
import os
import shutil
from nltk.corpus import stopwords
import pandas as pd
import torch
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import gensim
from gensim.corpora.dictionary import Dictionary
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import TensorDataset, DataLoader
from torchvision.transforms import ToTensor
import warnings 
from sklearn.linear_model import LogisticRegression
from PIL import Image
from IPython.display import clear_output
from typing import List
import nltk
import string
import pymorphy2
import codecs
from gensim.models import KeyedVectors
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
warnings.filterwarnings(action='ignore',category=UserWarning, module='gensim')  
warnings.filterwarnings(action='ignore',category=FutureWarning, module='gensim')  
warnings.filterwarnings(action='ignore',category=DeprecationWarning, module='gensim')
warnings.filterwarnings(action='ignore',category=DeprecationWarning, module='smart_open') 
warnings.filterwarnings(action='ignore',category=DeprecationWarning, module='sklearn')
warnings.filterwarnings(action='ignore',category=DeprecationWarning, module='scipy')    
warnings.filterwarnings(action='ignore',category=DeprecationWarning, module='pymorphy2')

In [5]:
nltk.download('stopwords')
nltk.download('punkt')
stopWordsRu = set(stopwords.words("russian")) 
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [23]:
def get_data() -> pd.DataFrame:
  dop = pd.read_csv('/content/drive/MyDrive/aiijc_sber/full_marked_dataset.csv')
  dop = dop[dop.ans!='неизвестно']
  mit_dict = {0:"животные",
              1: "музыка",
              2: "спорт",
              3: "литература"}
  data_texts = dop.text.to_list()
  data_ans = dop.ans.to_list()
  data = pd.DataFrame()
  data['text'] = data_texts
  data['ans'] = data_ans
  return data


In [24]:
data = get_data()

In [8]:
class Preparator():
  def __init__(self):
      self.morph = pymorphy2.MorphAnalyzer()
      self.tokenizer = nltk.WordPunctTokenizer()
      self.stopwords = set(line.strip() for line in codecs.open('/content/drive/MyDrive/aiijc_sber/all_stopwords.txt', "r", "utf_8_sig").readlines())
      self.bigram_mod = gensim.models.Phrases.load("/content/drive/MyDrive/aiijc_sber/bigram_model.pkl")

  def prepare_corp(self, news_list: List[str]):
      return [self.newstext2token(news_text) for news_text in news_list]

  def make_bigrams(self, doc):
      return self.bigram_mod[doc]

  def newstext2token(self, news_text: str):
      tokens = self.tokenizer.tokenize(news_text.lower())
      tokens_with_no_punct = [self.morph.parse(w)[0].normal_form for w in tokens if all(c in 'йцукенгшщзхъёфывапролджэячсмитьбю0123456789' for c in w)]
      tokens_base_forms = [w for w in tokens_with_no_punct if w not in self.stopwords]
      tokens_bigrammed = self.make_bigrams(tokens_base_forms)
      return tokens_bigrammed

In [25]:
prep = Preparator()
all_texts = data.text
print(len(all_texts))
all_tokens = []
for i in range(len(all_texts)):
  all_tokens.append(prep.newstext2token(all_texts[i]))
  if i%1000 == 0:
    print(i)
data['tokens'] = all_tokens

727
0


In [29]:
metrics = {}
for tokens in data.tokens:
  for word_num in range(len(tokens)):
    if tokens[word_num] == 'сколько' or tokens[word_num].isdigit():

      if word_num < len(tokens)-2 and tokens[word_num+2] in ['в', 'на', 'за']:
        metric = tokens[word_num+1] + '_' + tokens[word_num+2] + '_' + tokens[word_num+3] 
        if metric not in metrics:
          metrics[metric] = 0
        metrics[metric] += 1

      elif word_num < len(tokens)-1 and tokens[word_num+1].isalpha():
        metric = tokens[word_num+1]
        if metric not in metrics:
          metrics[metric] = 0
        metrics[metric] += 1



In [97]:
metrics = [" л ", " г ", " м ", " сек ", " ч ", " км ", " мл ", " кг ",' км/ч ', " кг/м ", " м/с ", " мин", "метр", "грамм", "штук", "час", "секунд", " руб", "килограмм", 'километр', "миллиметр", "сантиметр", "километров в час", "килограмм на метр"]

In [98]:
morph = pymorphy2.MorphAnalyzer()
verb_metr = {}
for tokens_ind in range(len(data)):
  for word in nltk.WordPunctTokenizer().tokenize(data.text[tokens_ind]):
    if morph.parse(word)[0].tag.POS == "VERB":
      # print(word)
      verb = word
  if verb:
    for metric in metrics:
      if metric in data.text[tokens_ind]:
        metr = morph.parse(metric)[0].normal_form
        if metr[-1] == 'а':
          metr = metr[:-1]
        verb_metr[morph.parse(verb)[0].normal_form] = metr.strip()

In [99]:
verb_metr

{'бежать': 'м',
 'болтать': 'мин',
 'быть': 'час',
 'вернуться': 'мин',
 'весить': 'килограмм',
 'выпить': 'л',
 'глядеть': 'л',
 'гореть': 'м',
 'давать': 'л',
 'дать': 'час',
 'делать': 'час',
 'длиться': 'час',
 'добавлять': 'час',
 'добираться': 'мин',
 'достигать': 'метр',
 'ехать': 'час',
 'жить': 'метр',
 'заготовить': 'килограмм',
 'занимать': 'час',
 'заплатить': 'руб',
 'засушить': 'кг',
 'затрачивать': 'мин',
 'заявить': 'час',
 'играть': 'мин',
 'иметь': 'килограмм',
 'испечь': 'час',
 'исчезнуть': 'час',
 'купить': 'руб',
 'лежать': 'килограмм',
 'мочь': 'г',
 'набрать': 'метр',
 'накопить': 'руб',
 'насушить': 'килограмм',
 'находиться': 'штук',
 'оказаться': 'килограмм',
 'определить': 'г',
 'остаться': 'час',
 'отремонтировать': 'штук',
 'пить': 'час',
 'подарить': 'руб',
 'подсчитать': 'кг',
 'поймать': 'штук',
 'пойти': 'руб',
 'положить': 'руб',
 'получать': 'грамм',
 'получиться': 'час',
 'понадобиться': 'метр',
 'поплыть': 'метр',
 'посадить': 'штук',
 'посвятить':

In [ ]:
{'бежать': 'метр',
 'болтать': 'мин',
 'быть': 'час',
 'вернуться': 'мин',
 'весить': 'кг',
 'выпить': 'литр',
 'гореть': 'мин',
 'давать': 'литр',
 'дать': 'час',
 'делать': 'час',
 'длиться': 'час',
 'добавлять': 'час',
 'добираться': 'мин',
 'достигать': 'метр',
 'ехать': 'час',
 'жить': 'метр',
 'заготовить': 'кг',
 'занимать': 'час',
 'заплатить': 'руб',
 'засушить': 'кг',
 'затрачивать': 'мин',
 'заявить': 'час',
 'играть': 'мин',
 'иметь': 'кг',
 'испечь': 'час',
 'исчезнуть': 'час',
 'купить': 'руб',
 'лежать': 'кг',
 'набрать': 'метр',
 'накопить': 'руб',
 'насушить': 'кг',
 'находиться': 'штук',
 'оказаться': 'кг',
 'остаться': 'час',
 'отремонтировать': 'штук',
 'пить': 'литр',
 'подарить': 'руб',
 'подсчитать': 'кг',
 'поймать': 'штук',
 'положить': 'руб',
 'получать': 'грамм',
 'получиться': 'час',
 'понадобиться': 'метр',
 'поплыть': 'метр',
 'посадить': 'штук',
 'посвятить': 'час',
 'поступить': 'час',
 'потратить': 'руб',
 'потребоваться': 'метр',
 'преодолеть': 'километр',
 'привезти': 'кг',
 'принести': 'час',
 'принимать': 'час',
 'принять': 'час',
 'пришить': 'метр',
 'пробежать': 'час',
 'проводить': 'час',
 'продать': 'час',
 'продолжаться': 'час',
 'проехать': 'метр',
 'пройти': 'метр',
 'пролететь': 'метр',
 'проплыть': 'час',
 'проскакать': 'километр',
 'прочитать': 'мин',
 'прыгать': 'метр',
 'пугаться': 'час',
 'разделить': 'час',
 'разнести': 'штук',
 'решить': 'час',
 'сгрызть': 'мин',
 'сделать': 'час',
 'сидеть': 'мин',
 'смотреть': 'сантиметр',
 'собирать': 'мин',
 'собрать': 'кг',
 'составлять': 'час',
 'сосчитать': 'штук',
 'спать': 'час',
 'стать': 'грамм',
 'стоить': 'руб',
 'стоять': 'руб',
 'съедать': 'кг',
 'съесть': 'кг',
 'увеличиться': 'час',
 'уйти': 'метр',
 'указывать': 'час',
 'уничтожить': 'грамм',
 'участвовать': 'час',
 'хватить': 'штук',
 'читать': 'страниц'}